In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# EXAMPLE CODE

# IMPORTS:
from wav_conversion import convert_mp3_to_wav
from textgrid_creation import create_textgrids
from custom_mfa_align import align
from formant_extraction import extract

# ARGUMENTS:
mp3_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/mp3-input"
in_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-input"
out_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-output"
home_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class" 
label = "Please call Stella. Ask her to bring these things with her from the store: \
Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for \
her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She \
can scoop these things into three red bags, and we will go meet her Wednesday at the train station."
acoustic_model = "english_us_arpa"
dictionary_model = "english_us_arpa"
filename="testing.csv"

# this converts mp3 files to wav files
# convert_mp3_to_wav(mp3_folder, in_folder, out_folder)
# this creates the textgrids
create_textgrids(label, in_folder)
# this aligns the textgrids
align(in_folder, out_folder, acoustic_model, dictionary_model)
# this calculates formant measurements
extract(out_folder, home_folder, filename)

TextGrid Creation Successful
Alignment Successful
Extraction Successful


In [3]:
import pandas as pd
from speech_modeling import h_model, h_input
raw_df = pd.read_csv("./gmu-formants.csv")

input_data = h_input(raw_df)
input_data.process('region')
input_data.select_features(selected_features=['F1mF2', 'duration'])
input_data.normalize(method='minmax')
input_data.select_places(places=['uk', 'texas', 'california', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(NUM_SAMPLES=5)

Fitting Model of type rforest
Modeling Sample 1 of 5


KeyboardInterrupt: 

In [ ]:
input_data.revert()
input_data.process('region')
input_data.select_features(selected_features=['F1', 'F2', 'F3', 'duration'])
input_data.normalize(method='z')
input_data.select_places(places=['uk', 'texas', 'california', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(NUM_SAMPLES=8)

In [ ]:
input_data.revert()
input_data.process('region')
input_data.select_features(selected_features=['F1mF2', 'duration'])
input_data.normalize(method='minmax')
input_data.select_places(places=['northcarolina', 'texas', 'california', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="northcarolina")
m.fit(NUM_SAMPLES=5)

In [ ]:
input_data.revert()
input_data.process('region')
input_data.select_features(selected_features=['F1mF2', 'duration'])
input_data.normalize(method='minmax')
input_data.select_places(places=['northcarolina', 'newyork'])

model_data = input_data.input_df
model_features = ['age', 'gender'] + list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="northcarolina")
m.fit(NUM_SAMPLES=5)